# Quantum Proof of Work: q-circuit with parametric quantum gates
#### code developed by Mikhail Shalaginov (mys@mit.edu) and Michael Dubrovsky (mike@powx.org)

Proof of work (PoW) is the mechanism allowing formation of decentralized cryptocurrency blockchains (e.g. Bitcoin, Etherium, etc). Here we introduce a quantum computing part to a traditional mining cycle in a similar way as it has been proposed earlier for <a href="https://assets.pubpub.org/xi9h9rps/01581688887859.pdf">optical PoW</a>. The QPoW cycle is illustrated on a schematic below. 

The input ('text') is a concatenated string containing: nonce, transaction data and a hash-string from a previous run. Nonce is an integer starting from 0 value and incrementing by 1 with each cycle, transaction data is a ledger entry, e.g. <font color='green'>'Schroedinger paid Einstein 1 qBTC'</font>. By SHA3 algorithm the 'text' (e.g. <font color='green'>'4Schroedinger paid Einstein 1 qBTC04ca1a782621a440d03b5d87ecff8b68e2cc6124f57957b49a76bca91dede3a81'</font>) is transformed into a 256-bit hash-string (<font color='green'>'e1e5575da3a9e86da135552facddcc1ff44dd26502d0bc2b22961383f8b187ca'</font>) that is pushed through the QPoW stages. After all of the computations, we have a 256-bit hash-string as an output(<font color='green'>'f307b3db12a649563831e3e1328c3c7a5b15ee541afaab563727cb992cf9d1ca'</font>). If the output satisfies the conditions set by the task difficulty, i.e. the output string has a certain number of zeros, then the block is reported to be successfully found. Otherwise, the output hash feeds back into the input 'text' and the nonce is increased by 1. When the block is finally found, it goes through a verification test performed by a quantum simulator ran on a classical computer.

In this demo we demonstrate the construction of a block-chain composed of several elements (5 in this example) by computing the quantum algorithm on a 4-qubit IBM Q node (<a href="https://quantum-computing.ibm.com/">ibmq_lima</a>).

<img src="images/QPoW_schematic2.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="1200 px" align="center">

In [ ]:
# import modules for SHA
import sys
import hashlib
import numpy as np
import random

if sys.version_info < (3, 6):
	import sha3

# import modules for Qiskit
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer, Aer, IBMQ, execute, schedule
from qiskit.compiler import transpile
from qiskit.tools.visualization import plot_histogram
from qiskit.transpiler import PassManager
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer import AerSimulator
# from azure.quantum.qiskit import AzureQuantumProvider

# import other useful packages 
from numpy import pi
from datetime import date
import time

# settings for IBM Q backends
# TOKEN = ''
# IBMQ.save_account(TOKEN)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
qpu_name = 'ibmq_lima'
simulator_name = 'qasm_simulator'
qpu_backend = provider.get_backend(qpu_name)

# create data-storage files
today = date.today()
date = today.strftime("%Y%m%d")
filepath = '' #specify data storage folder
filename_results = filepath + date + '_results_IBM_ch7_exp.txt'
filename_log = filepath + date + '_log_IBM5_ch7_exp.txt'
filename_bc = filepath + date + '_block_chain7_exp.txt'

# input parameters
previous_hash = '' #empty hash
block_number = 0
numberOfBlocks = 5
errors = 0 #number of 'False' nonces
difficulty = 1 #number of prefix zeros (hash-problem difficulty)
transactions ='Schroedinger paid Einstein 1 qBTC'
QC_switch = 1 #0 - simulator, 1 - real QC
MAX_NONCE = 100000000000

# loop for building a blockchain
counter = 0
while counter < numberOfBlocks:

    with open(filename_log, 'a') as o:
        o.write('mining started \n')
        o.write('difficulty:'+ str(difficulty)+ '\n')
        o.write('previous_hash:' + str(previous_hash)+ '\n')
        o.write('transactions:' + str(transactions) +'\n')
        o.write('block_number:' + str(block_number) + '\n')
        o.write('QC_switch:' + str(QC_switch) + '\n\n')

    with open(filename_results, 'a') as o:
        o.write('mining started \n')
        o.write('difficulty:'+ str(difficulty)+ '\n')
        o.write('previous_hash:' + str(previous_hash)+ '\n')
        o.write('transactions:' + str(transactions) +'\n')
        o.write('block_number:' + str(block_number) + '\n')
        o.write('QC_switch:' + str(QC_switch) + '\n\n')

    if QC_switch == 1:
        with open(filename_results, 'a') as o:
            o.write('IBM node:'+ qpu_name)
    else:
        with open(filename_results, 'a') as o:
            o.write('IBM node:'+ simulator_name)

### Mining functions

    # QPoW cycle: function takes the input text (block number, nonce,...) and pushes it through once 
    def qPoW(text, QC_switch, quantum_circuit, nonce):

        hashIn = hashlib.sha3_256(text.encode("ascii")).hexdigest() # hashing the 'text' input
        #string-type output
        
        print ('hashIn-hex:', hashIn, 'length:', len(hashIn))
        with open(filename_log, 'a') as o:
            o.write('hashIn-hex:' + str(hashIn)+ '\n')

        # convert hashIn(hex) to hashIn_bin(binary)
        scale = 16 #hex base
        hashIn_bin = bin(int(hashIn, scale))[2:].zfill(len(hashIn)*4)
        print ('hashIn-binary:', str(hashIn_bin), 'length:', len(hashIn_bin))
        with open(filename_log, 'a') as o:
            o.write('hashIn-binary:' + str(hashIn_bin)+ '\n')

        #switch to enable quantum simulator or quantum computer
        if QC_switch == 0:
            [qstate_bin, comp_time] = sim_quantum_operation(quantum_circuit, hashIn_bin, nonce)
        if QC_switch == 1: 
            [qstate_bin, comp_time] = exp_quantum_operation(quantum_circuit, hashIn_bin, nonce, qpu_backend) #qstate is a 256 binary number

        print('qblock-output:', str(qstate_bin), 'length:', len(qstate_bin))

        xor_bin = xor(str(hashIn_bin), str(qstate_bin), 256)
        hashOut = hashlib.sha3_256(xor_bin.encode("ascii")).hexdigest()

        print('XOR:', str(xor_bin), 'length:', len(xor_bin))
        with open(filename_log, 'a') as o:
            o.write('XOR:'+ str(xor_bin) + 'length:'+ str(len(xor_bin)) + '\n')

        print('hashOut-hex:', str(hashOut), 'length:', len(hashOut), '\n\n')

        return [hashOut, comp_time]

    #mining based on multiple qPoW cycles, the function runs untill it finds the nonce satisfying the difficulty conditions
    def mine(block_number, transactions, previous_hash, prefix_zeros, QC_switch):

        prefix_str = '0'*prefix_zeros #sets the difficulty, in hex format, bin: multiply by 4
        
        start = time.time()
        comp_time_block = 0
        
        for nonce in range(MAX_NONCE):
            text = str(block_number) + transactions + previous_hash + str(nonce) #hash input
            print ('text:', text, '\n'  )
            [new_hash, comp_time] = qPoW(text,QC_switch, quantum_circuit, nonce)

            #write to log file
            with open(filename_log, 'a') as o:
                o.write('nonce:'+ str(nonce)+'\n')
                o.write('hashOut:'+ str(new_hash)+'\n\n')
            comp_time_block+=comp_time
            
            if new_hash.startswith(prefix_str):
                break

        print(f"qPoW completed. Successfully mined a block with a nonce value:{nonce}")
        with open(filename_log, 'a') as o:
            o.write('qPoW completed. Successfully mined a block with a nonce value' +str(nonce)+'\n')
        with open(filename_results, 'a') as o:
            o.write('qPoW completed.' +str(nonce)+'\n')
        with open(filename_bc, 'a') as o:
            o.write('computational time for mining a block:' +str(comp_time_block)+'\n')
        
        total_time = str((time.time() - start))
        print(f"mining ended. mining time: {total_time} seconds")
        print('final hash:', new_hash)
        print('suitable nonce:', nonce)
        
        return [new_hash, nonce, comp_time]

        raise BaseException(f"Couldn't find correct has after trying {MAX_NONCE} times")

    ### Quantum operations

    # converting hashIn_bin to a bit string to pass thru a quantum processor
    def break_up_4bit_values(hashIn_bin):

        array_4_bit_values = []
        i = 0

        while i < 64 : 
          four_bits = hashIn_bin[2+4*i:2+4*i+4]
          array_4_bit_values.append(four_bits)
          i = i + 1
        print("hashIn binary split into 4bit bins:", array_4_bit_values)
        return array_4_bit_values
    
    # setting the quantum circuit:
    def quantum_circuit(q_par, n_qreg = 4, n_creg = 4, circ_layer = 1):
        qreg_q = QuantumRegister(n_qreg, 'q')
        creg_c = ClassicalRegister(n_creg, 'c')
        circuit = QuantumCircuit(qreg_q, creg_c)

        #circuit 6 modified
        for i in range(circ_layer):
            circuit.rx(q_par[0]*pi/8, qreg_q[0])
            circuit.rx(q_par[1]*pi/8, qreg_q[1])
            circuit.rx(q_par[2]*pi/8, qreg_q[2])
            circuit.rx(q_par[3]*pi/8, qreg_q[3])
            circuit.rz(q_par[4]*pi/2, qreg_q[0])
            circuit.rz(q_par[5]*pi/8, qreg_q[1])
            circuit.rz(q_par[6]*pi/8, qreg_q[2])
            circuit.rz(q_par[7]*pi/8, qreg_q[3])
            circuit.crx(q_par[8]*pi/8, qreg_q[3], qreg_q[2])
            circuit.crx(q_par[9]*pi/8, qreg_q[3], qreg_q[1])
            circuit.crx(q_par[10]*pi/8, qreg_q[3], qreg_q[0])
            circuit.crx(q_par[11]*pi/8, qreg_q[2], qreg_q[3])
            circuit.crx(q_par[12]*pi/8, qreg_q[2], qreg_q[1])
            circuit.crx(q_par[13]*pi/8, qreg_q[2], qreg_q[0])
            circuit.crx(q_par[14]*pi/8, qreg_q[1], qreg_q[3])
            circuit.crx(q_par[15]*pi/8, qreg_q[1], qreg_q[2])
            circuit.crx(q_par[16]*pi/8, qreg_q[1], qreg_q[0])
            circuit.rz(q_par[17]*pi/8, qreg_q[3])
            circuit.rz(q_par[18]*pi/8, qreg_q[2])
            circuit.rz(q_par[19]*pi/8, qreg_q[0])
            circuit.rz(q_par[20]*pi/8, qreg_q[1])
            circuit.crx(q_par[21]*pi/8, qreg_q[0], qreg_q[3])
            circuit.crx(q_par[22]*pi/8, qreg_q[0], qreg_q[2])
            circuit.crx(q_par[23]*pi/8, qreg_q[0], qreg_q[1])
            circuit.rx(q_par[24]*pi/8, qreg_q[3])
            circuit.rx(q_par[25]*pi/8, qreg_q[2])
            circuit.rx(q_par[26]*pi/8, qreg_q[0])
            circuit.rx(q_par[27]*pi/8, qreg_q[1])

        #measurements of all qubits
        for i in range(len(qreg_q)):
            circuit.measure(qreg_q[i], creg_c[i])
        
        return circuit
        
        
    # real(experimental) quantum computor run
    def exp_quantum_operation(quantum_circuit, hashIn, nonce, qpu_backend):

        #input hashIn string
        fourbit_array = break_up_4bit_values(hashIn)
        q_par = [int(fourbit_array[i],2) for i in range(len(fourbit_array)-1)] #throwing away the last string element
        circuit = quantum_circuit(q_par)
        
        #for IBMQ machines the circuit needs to go thru the transpilation (expression in basic gates)
        transpiled_circuit = transpile(circuit, qpu_backend, seed_transpiler=13)

        qpu_job = qpu_backend.run(transpiled_circuit, shots=20000)
        job_id = qpu_job.job_id()
        print("Job id", job_id)

        # Monitor job progress and wait until complete:
        job_monitor(qpu_job)

        # Get the job results (this method also waits for the Job to complete):
        results = qpu_job.result()
        print(results)
        
        with open(filename_log, 'a') as o:
            o.write(str(results) + '\n')

        with open(filename_results, 'a') as o:
            o.write("time taken: {} sec".format(results.time_taken) + '\n')
        
        comp_time = results.time_taken
        
        counts = {format(n, "03b"): 0 for n in range(8)}
        counts.update(results.get_counts(circuit))
        print(counts)

        # plot histograms
        hist_filename = filepath + 'exp_histogram' + job_id
        plot_histogram(counts, filename = hist_filename, figsize=(7, 5))

        #draw the quantum circuit
        drawC_filename = filepath + 'exp_q_circuit' + job_id
        drawTC_filename = filepath + 'exp_q_Tcircuit' + job_id
        circuit.draw('mpl', fold=0, filename = drawC_filename)
        transpiled_circuit.draw('mpl',fold=0, filename = drawTC_filename)

        #picking up the maximally probable state
        max_state = max(counts, key=counts.get)
        max_state256 = max_state

        for i in range(256 - len(max_state)):
            max_state256+='0' 

        with open(filename_results, 'a') as o:
            o.write(str(job_id) + ',' + str(max_state)+',')

        return [max_state256, comp_time] #4bit vector

    # quantum simulator run
    def sim_quantum_operation(quantum_circuit, hashIn, nonce):

        #input hashIn string
        fourbit_array = break_up_4bit_values(hashIn)
        q_par = [int(fourbit_array[i],2) for i in range(len(fourbit_array)-1)] #throwing away the last string element
        circuit = quantum_circuit(q_par)

        backend = BasicAer.get_backend(simulator_name) # run on local simulator by default 

        job = execute(circuit, backend, shots=20000)

        # Monitor job progress and wait until complete:
        job_monitor(job)

        # Get the job results (this method also waits for the Job to complete):
        results = job.result()
        print(results)

        with open(filename_log, 'a') as o:
            o.write(str(results) + '\n')

        with open(filename_results, 'a') as o:
            o.write("time taken: {} sec".format(results.time_taken) + '\n')
        
        comp_time = results.time_taken
        counts = results.get_counts(circuit)

        #plot histograms
        hist_filename = filepath + 'sim_histogram' + str(nonce)
        plot_histogram(counts, filename = hist_filename, figsize=(7, 7))

        #draw the quantum circuit
        drawC_filename = filepath + 'sim_q_circuit' + str(nonce)
        circuit.draw('mpl', fold=0, filename = drawC_filename)

        #picking up the maximally probable state
        max_state = max(counts, key=counts.get)
        print ('max_state:', max_state)
        max_state256 = max_state

        for i in range(256 - len(max_state)):
            max_state256+='0' 

        with open(filename_results, 'a') as o:
            o.write('max_state = '+str(max_state)+', '+'nonce = '+str(nonce)+', ' + '\n')


        return [max_state256, comp_time] #4bit vector

    def xor(a, b, n): #a,b - strings, n - length of the XOR output
        ans = ""

        # Loop to iterate over the
        # Binary Strings
        for i in range(n):

            # If the Character matches
            if (a[i] == b[i]):
                ans += "0"
            else:
                ans += "1"
        return ans


    ### Execute QPoW
    [new_hash, nonce, comp_time] = mine(block_number,transactions, previous_hash, difficulty, QC_switch)
    
    ### Verify with the quantum simulator

    def verify(nonce, block_number, transactions, previous_hash, prefix_zeros, quantum_circuit, QC_switch):
        prefix_str = '0'*prefix_zeros

        print('nonce:', nonce)
        text = str(block_number) + transactions + previous_hash + str(nonce)
        print('text:', text)
        [new_hash, ver_time] = qPoW(text, QC_switch, quantum_circuit, nonce)

        if new_hash.startswith(prefix_str):
            print(f"Verified: nonce {nonce}\n") 
            with open(filename_results, 'a') as o:
                o.write('Verified' + '\n')
            with open(filename_log, 'a') as o:
                o.write('Verified' + '\n')
            return [True, new_hash, ver_time]
        else:
            print(f"False:{nonce}\n")
            with open(filename_results, 'a') as o:
                o.write('False' + '\n')
            with open(filename_log, 'a') as o:
                o.write('False' + '\n')
            return [False, previous_hash, ver_time]
    
    [check, new_hash, ver_time] = verify(nonce, block_number, transactions, previous_hash, difficulty, quantum_circuit, QC_switch = 0)
    
    if check == True: 
        previous_hash = new_hash
        block_number+= 1
        counter+= 1 
        with open(filename_bc, 'a') as o:
            o.write('block_number:' + str(block_number-1) + '\n')
            o.write('nonce:' + str(nonce)+ '\n')
            o.write('previous_hash:' + str(previous_hash)+ '\n')
            o.write('transactions:' + str(transactions) +'\n')
            o.write('errors:' + str(errors) +'\n')
            o.write('time for verifying a block:' +str(ver_time)+'\n\n')
        
    else:
        previous_hash = previous_hash
        block_number = block_number
        errors += 1
        print('number of errors:', errors)
        

text: 0Schroedinger paid Einstein 1 qBTC0 

hashIn-hex: 0876a34b05a4b00141da5b044fff88fe3e81a92842fb42ba1727078a303e6c67 length: 64
hashIn-binary: 0000100001110110101000110100101100000101101001001011000000000001010000011101101001011011000001000100111111111111100010001111111000111110100000011010100100101000010000101111101101000010101110100001011100100111000001111000101000110000001111100110110001100111 length: 256
hashIn binary split into 4bit bins: ['0010', '0001', '1101', '1010', '1000', '1101', '0010', '1100', '0001', '0110', '1001', '0010', '1100', '0000', '0000', '0101', '0000', '0111', '0110', '1001', '0110', '1100', '0001', '0001', '0011', '1111', '1111', '1110', '0010', '0011', '1111', '1000', '1111', '1010', '0000', '0110', '1010', '0100', '1010', '0001', '0000', '1011', '1110', '1101', '0000', '1010', '1110', '1000', '0101', '1100', '1001', '1100', '0001', '1110', '0010', '1000', '1100', '0000', '1111', '1001', '1011', '0001', '1001', '11']
Job id 6211dc0006362898792d7c1d
Job S